# Establishing a method / procedure

## import the file in its original version xml
### continuing... 

In [1]:
with open ("db_savoirs/BNU_04_Gfrereis.xml", "r", encoding="utf_8") as myfile:
    data = myfile.read()
    print(data)

<TEI xmlns="http://www.tei-c.org/ns/1.0" xmlns:aid="http://ns.adobe.com/AdobeInDesign/4.0/" xmlns:aid5="http://ns.adobe.com/AdobeInDesign/5.0/" xmlns:dcr="http://www.isocat.org/ns/dcr" xmlns:examples="http://www.tei-c.org/ns/Examples" xmlns:hfp="http://www.w3.org/2001/XMLSchema-hasFacetAndProperty" xmlns:loext="urn:org:documentfoundation:names:experimental:office:xmlns:loext:1.0" xmlns:marcrel="http://www.loc.gov/loc.terms/relators" xmlns:mets="http://www.loc.gov/METS/" xmlns:mods="http://www.loc.gov/mods/v3" xmlns:ns="http://www.tei-c.org/ns/1.0" xmlns:trans="http://docbook.org/ns/transclusion" xmlns:xhtml="http://www.w3.org/TR/xhtml/strict" xmlns:xi="http://www.w3.org/2001/XInclude" xmlns:xila="http://www.w3.org/2001/XInclude/local-attributes" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns:xs="http://www.w3.org/2001/XMLSchema" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" lock="ndufournaud" time="2020-10-07T12:20:56.726+02:00" xsi:schemaLocation="http://www.tei-c.org/ns/1.0

## transform the file from xml to txt

In [2]:
import re

text = re.sub('<[^<]+>', "", open("db_savoirs/BNU_04_Gfrereis.xml").read())
with open("output_savoirs/BNU_04_Gfrereis.txt", "w") as f:
    f.write(text)

## open the file to verify if it works and if it was correctly done 

In [3]:
with open ("output_savoirs/BNU_04_Gfrereis.txt", "r", encoding="utf_8") as myfile:
    data = myfile.read()
    print(data)


  
    
      
        Dé-construire en exposant : Ernst Jünger,
        travailleur au bord de l’abîme aux Archives littéraires allemandes de
        Marbach
        La Revue de la BNU no 3
        
          Heike
          Gfrereis
          052549275
        
        
          Aude
          Therstappen
          142817252
        
        EHESS
        Projet Savoirs
        
          encodage et supervision
          
            Nicole
            Dufournaud
          
        
        
          supervision
          
            Axel
            Le Roy
          
        
      
      
        Première édition numérique, Paris, 2020 dans le cadre du projet Savoirs.
      
      
        EHESS
        
          54 bd Raspail
          75006 Paris
          FRANCE
        
        
          Creative
          Commons Attribution
          BNU
          BY-NC-SA Attribution - Pas d’Utilisation Commerciale - Partage dans
          les Mêmes Conditions
        
      
      
 

## Starting SpaCy  procedure ##
### import spacy and its model ###

In [4]:
import spacy
nlp = spacy.load("fr_core_news_md")

### import the file in its version .txt

In [5]:
with open ("output_savoirs/BNU_04_Gfrereis.txt", "r", encoding="utf_8") as myfile:
    data = myfile.read()

### apply SpaCy's model to the file

In [6]:
doc = nlp(data)

###  I prefer print it to verify if the file is loaded

### Highlighting the results 

In [7]:
from spacy import displacy
options = {"ents": ["LOC", "FAC", "GPE"]}
displacy.render(doc, style = "ent",jupyter = True, options=options)

### apply a python function to classify the entities catches from the file

In [8]:
def show_ents(doc): 
    if doc.ents: 
        for ent in doc.ents: 
            print(ent.text+' - ' +str(ent.start_char) +' - '+ str(ent.end_char) +' - '+ent.label_+ ' - '+str(spacy.explain(ent.label_))) 
            print("\n")             
show_ents(doc)

Ernst Jünger - 52 - 64 - PER - Named person or family.


Archives littéraires allemandes - 109 - 140 - MISC - Miscellaneous entities, e.g. events, nationalities, products or works of art


Marbach - 152 - 159 - LOC - Non-GPE locations, mountain ranges, bodies of water


Revue de la - 171 - 182 - MISC - Miscellaneous entities, e.g. events, nationalities, products or works of art


Heike
          Gfrereis - 211 - 235 - MISC - Miscellaneous entities, e.g. events, nationalities, products or works of art


Aude - 284 - 288 - LOC - Non-GPE locations, mountain ranges, bodies of water


Therstappen - 299 - 310 - LOC - Non-GPE locations, mountain ranges, bodies of water


EHESS - 348 - 353 - ORG - Companies, agencies, institutions, etc.


Projet Savoirs - 362 - 376 - ORG - Companies, agencies, institutions, etc.


Nicole
            Dufournaud - 443 - 472 - MISC - Miscellaneous entities, e.g. events, nationalities, products or works of art


Axel - 547 - 551 - MISC - Miscellaneous entities, e.

### display the results in the file as highlight

In [9]:
from spacy import displacy
displacy.render(doc, style = "ent",jupyter = True)

### Create a file jsonl with the results 

In [10]:
import json
from spacy.tokens import Span
import codecs

with codecs.open("output_annotations/BNU_04_Gfrereis_an.jsonl", 'w', encoding='utf8') as f:
    for sent in doc.sents:
        text = sent.text
        if(text.strip() != '' and not text.startswith("\n")): #pour enlever quelques lignes avec encodage problématique
            labels = []
            if (len(sent.ents) > 0):
                for ent in sent.ents:
                    labels.append([ent.start_char-ent.sent.start_char, ent.end_char-ent.sent.start_char, ent.label_])
            #print(text)
            if (len(labels) > 0):
                sentence = {"text": text, "labels": labels}    
            else:
                sentence = {"text": text}
            #print(json_string)
            json_string = json.dumps(sentence, ensure_ascii=False)
            f.write(json_string)
            f.write("\n")
f.close()
print("fichier créé : BNU_04_Gfrereis_an.jsonl")

fichier créé : BNU_04_Gfrereis_an.jsonl


### Print the file to verify the annotations

In [12]:
TRAIN_DATA = []

with codecs.open("output_annotations/BNU_04_Gfrereis_an.jsonl", "r", encoding="utf8") as f:
    lines = f.readlines()
    for line in lines:
        line = json.loads(line)
        #print(line)
        if "labels" in line:
            line["entities"] = line.pop("labels")
        else:
            line["entities"] = []
        ents = []
        for entity in line["entities"]:
            ents.append((entity[0], 
                         entity[1], 
                         entity[2]))
        TRAIN_DATA.append({"text": 
                           line["text"], 
                           "entities": ents})
        #json_string = json.dumps({"text": line["text"], "entities": ents}, ensure_ascii=False)
f.close()

print(TRAIN_DATA)

[{'text': 'Dé-construire en exposant\xa0: Ernst Jünger,\n        travailleur au bord de l’abîme aux Archives littéraires allemandes de\n        Marbach', 'entities': [(28, 40, 'PER'), (85, 116, 'MISC'), (128, 135, 'LOC')]}, {'text': 'Aude\n          Therstappen', 'entities': [(0, 4, 'LOC'), (15, 26, 'LOC')]}, {'text': '142817252', 'entities': []}, {'text': 'Nicole\n            Dufournaud', 'entities': [(0, 29, 'MISC')]}, {'text': 'supervision', 'entities': []}, {'text': 'Axel\n            ', 'entities': [(0, 4, 'MISC')]}, {'text': 'Le Roy', 'entities': [(0, 6, 'MISC')]}, {'text': 'dans le cadre du projet Savoirs', 'entities': []}, {'text': '.', 'entities': []}, {'text': '54 bd Raspail\n          ', 'entities': [(6, 13, 'LOC')]}, {'text': '75006 Paris', 'entities': [(6, 11, 'LOC')]}, {'text': 'FRANCE', 'entities': []}, {'text': 'Creative\n          ', 'entities': [(0, 19, 'MISC')]}, {'text': 'Commons', 'entities': []}, {'text': 'Attribution\n          BNU\n          ', 'entities': [(22,